# Imports

In [178]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense
from keras.applications.vgg16 import preprocess_input, VGG16 
from keras.optimizers import adam_v2
from keras.metrics import categorical_crossentropy

# Loading images


First, we store the path to our training images and the classes inside a list

In [179]:
train_path = "data/train"
test_path = "data/test"
classes = ["1", "2", "3", "4"]

Then, we will use our images to generate some augmented images to increase our dataset

In [180]:
image_generator = ImageDataGenerator(preprocessing_function=preprocess_input,
                                     rotation_range=5,
                                     height_shift_range=0.1,
                                     width_shift_range=0.1,
                                     shear_range=0.15,
                                     horizontal_flip=True,
                                     vertical_flip=True)

train_images = image_generator.flow_from_directory(train_path,
                                                   target_size=(224, 224), 
                                                   classes=classes, 
                                                   batch_size=10)

images = np.ndarray(shape=(0,224,224,3))
labels = np.ndarray(shape=(0,4))
for i in range(4 * int(train_images.n / train_images.batch_size)):
    imgs, lbls = next(train_images)
    images = np.concatenate((images, imgs))
    labels = np.concatenate((labels, lbls))

Found 240 images belonging to 4 classes.


# Building our model


First, we will initialize a pre-trained model, so we will not have to train a full model ourselves

In [181]:
vgg16_model = VGG16()

Then, we will copy its layers but the last one, which we will subtitute with our own output layer of 4 units, according to our number of classes

In [182]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    layer.trainable = False
    model.add(layer)
model.add(Dense(4, activation="softmax"))

model.compile(optimizer=adam_v2.Adam(learning_rate=1e-3),
              loss=categorical_crossentropy,
              metrics=["accuracy"])

Finally, its time to train our model!

In [ ]:
history = model.fit(images, labels, 
                    validation_split=0.2,
                    epochs=10, 
                    verbose=2, 
                    batch_size=train_images.batch_size)   

Let's see how our model did

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

# Saving our model

Let's save our model in the path below

In [186]:
model_path = "/content/drive/MyDrive/Models/nyc_floor_plan_classiffier.pkl"

In [ ]:
with open(model_path, "wb") as model_file:
    pickle.dump(model, model_file)

# Reloading the model

Let's reload the model we saved in the previous section

In [188]:
with open(model_path, "rb") as model_file:
    model_reloaded = pickle.load(model_file)

We will make sure that the model was loaded correctly

In [189]:
assert all([np.array_equal(mrw, mw) for mrw, mw in zip(model_reloaded.get_weights(), model.get_weights())])

# Predicting an image class

We will create a predict function that returns the appropriate class, given an image of a floor plan

In [190]:
def predict(image_path):
    with open(model_path, "rb") as model_file:
        model_reloaded = pickle.load(model_file)

    image = load_img(image_path, target_size=(224,224))
    image = img_to_array(image)
    image = image.reshape((1,) + image.shape)

    return np.argmax(model_reloaded.predict(image)) + 1

First, let's load some test images

In [ ]:
test_images = ImageDataGenerator(preprocessing_function=preprocess_input).flow_from_directory(test_path,
                                                       target_size=(224, 224), 
                                                       classes=classes, 
                                                       batch_size=10)

images = np.ndarray(shape=(0,224,224,3))
labels = np.ndarray(shape=(0,4))
for i in range(int(test_images.n / test_images.batch_size)):
    imgs, lbls = next(test_images)
    images = np.concatenate((images, imgs))
    labels = np.concatenate((labels, lbls))

Then, we will evaluate our predictions

In [ ]:
predictions = model.predict(images)
predictions = np.argmax(predictions, axis=-1)
labels = np.argmax(labels, axis=-1)

cm = confusion_matrix(y_true=labels, 
                      y_pred=predictions)

print(cm)

In [ ]:
print(f"Prediction rate: {(labels == predictions).mean() * 100}%")